<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_fillable_form_in_pdf_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a fillable form in PDF format

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [12]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now that `borb` is installed, we can perform the basic imports;

In [13]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout

The following code represents the basic steps on creating a PDF document using `borb`:

- Creating an empty `Document`
- Creating an empty `Page`
- Appending the `Page` to the `Document`
- Creating a `PageLayout` that is responsible for handling the flow of content (here we'll use `SingleColumnLayout`)
- Adding content to the `PageLayout`
- Persisting the `Document` to disk

In [14]:
# create empty Document
pdf = Document()

# create empty Page
page = Page()

# add Page to Document
pdf.add_page(page)

# create PageLayout
layout: PageLayout = SingleColumnLayout(page)

The next part is actually adding the content of the form.
For this example we're going to create a form with some basic user-information questions.

- Name
- Surname
- Gender
- Place of residence
- Nationality

To ensure everything is laid out just right, we're going to add this content to a `Table`. The left column will contain the field name (e.g. "name", "surname"), the right column will contain the fields to be filled in.

In [15]:
# new import(s)
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.forms.text_field import TextField
from borb.pdf.canvas.color.color import HexColor
from decimal import Decimal
from borb.pdf.canvas.layout.layout_element import Alignment
from borb.pdf.canvas.layout.forms.drop_down_list import DropDownList

# Let's start by adding a heading
layout.add(Paragraph("Patient Information:", font="Helvetica-Bold"))

# Use a table to lay out the form
table: FixedColumnWidthTable = FixedColumnWidthTable(number_of_rows=5, number_of_columns=2)

# Name
table.add(Paragraph("Name : ", horizontal_alignment=Alignment.RIGHT, font_color=HexColor("56cbf9")))
table.add(TextField(value="Doe", font_color=HexColor("56cbf9"), font_size=Decimal(20)))

# Surname
table.add(Paragraph("Surname : ", horizontal_alignment=Alignment.RIGHT, font_color=HexColor("56cbf9")))
table.add(TextField(value="John", font_color=HexColor("56cbf9"), font_size=Decimal(20)))

We're going to model gender as a dropdown list, from which the reader can choose one of four options:

- Female
- Male
- Other
- Prefer not to disclose

Let's see how that translates to `borb`

In [16]:
# Gender
table.add(Paragraph("Gender : ", horizontal_alignment=Alignment.RIGHT))
table.add(DropDownList(
    possible_values=[
                    "Female",
                    "Male",
                    "Other",
                    "Prefer not to disclose",
                    ]
))

We could do a similar thing for country of residence and nationality, but it would involve having to find a list of all countries in the world and manually creating the array to pass to the constructor of `DropDownList`.

Because this particular field (a list of all countries) is such a common requirement, `borb` comes pre-loaded with the class `CountryDropDownList`.

Let's see it in action;


In [17]:
# new import(s)
from borb.pdf.canvas.layout.forms.country_drop_down_list import CountryDropDownList

# Country of Residence
table.add(Paragraph("Country of Residence : ", horizontal_alignment=Alignment.RIGHT))
table.add(CountryDropDownList(value="Belgium"))

# Nationality
table.add(Paragraph("Nationality : ", horizontal_alignment=Alignment.RIGHT))
table.add(CountryDropDownList(value="Belgium"))

Now we can finally add the `Table` to our `PageLayout`

In [18]:
# set some properties on the table to make the layout prettier
table.set_padding_on_all_cells(Decimal(5), Decimal(5), Decimal(5), Decimal(5))
table.no_borders()

# adding Table to PageLayout
layout.add(table)

Now let's add a (nonsense) data protection policy

In [19]:
# data protection policy
layout.add(Paragraph("Data Protection Policy", 
                     font="Helvetica-Bold"))

# dummy text
layout.add(Paragraph(
    """
    ** Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
    Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
    Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
    Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
    """,
    font="Helvetica-Oblique"
))

Let's wrap things up by adding a footer. For now, we'll just add a rectangle filled in the accent color, at the bottom of the page. Nothing too fancy.

In [20]:
# new import(s)
import typing
from borb.pdf.canvas.geometry.rectangle import Rectangle
from borb.pdf.page.page_size import PageSize
from borb.pdf.canvas.line_art.line_art_factory import LineArtFactory
from borb.pdf.canvas.layout.shape.shape import Shape

ps: typing.Tuple[Decimal, Decimal] = PageSize.A4_PORTRAIT.value
r: Rectangle = Rectangle(Decimal(0), Decimal(32), ps[0], Decimal(8))
Shape(points=LineArtFactory.rectangle(r), stroke_color=HexColor("56cbf9"), fill_color=HexColor("56cbf9")).layout(page, r)

Lastly, we can store the `Document` we created using the `PDF` class.

In [21]:
# new import(s)
from borb.pdf.pdf import PDF

# store
with open("output.pdf", "wb") as out_file_handle:
    PDF.dumps(out_file_handle, pdf)